In [0]:
import delta

def tabel_exists(catalog,database, tableName):
    
    count = (spark.sql(f"SHOW TABLES FROM {catalog}.{database}")
    .filter(f"database = '{database}' AND tableName = '{tableName}'")
    .count())

    return count == 1

In [0]:

schema = 'sistema_pontos'
catalog = 'bronze'
table = dbutils.widgets.get('tablename')
table_cdc = dbutils.widgets.get('tablename_cdc') ## tabela que contem o cdc
id_field = dbutils.widgets.get('id_field')

In [0]:
if not tabel_exists(catalog,schema,table):

  print("Tabela nao existente, criando com full_load")


  df_full = spark.read.format("parquet").load(f"/Volumes/raw/{schema}/full_load/{table}/")
  (df_full.coalesce(1)
  .write
  .format("delta")
  .mode("overwrite")
  .saveAsTable(f"{catalog}.{schema}.{table}"))

else:
  print("Tabela ja existente, ignorando full_load")

In [0]:
(spark.read
 .format("parquet")
 .load(f"/Volumes/raw/sistema_pontos/cdc/{table_cdc}/")
 .createOrReplaceTempView(f"view_{table}"))

In [0]:
query = f'''
    SELECT *
    FROM view_{table}
    QUALIFY row_number() OVER (PARTITION BY {id_field} ORDER BY _cdc_ts DESC) = 1
'''

df_cdc_unique = spark.sql(query)
# df_cdc_unique.display()

In [0]:
bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{table}")


In [0]:
(
    bronze.alias("b")
      .merge(df_cdc_unique.alias("d"), f"b.{id_field} = d.{id_field}")
      .whenMatchedDelete(condition = "d.op = 'D'")
      .whenMatchedUpdateAll(condition = "d.op = 'U'")
      .whenNotMatchedInsertAll(condition = "d.op = 'I' or d.op = 'U'")
      .execute()
)
